In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [3]:
import pandas as pd

In [4]:
sums_pd = pd.read_csv("stockholm/wikipedia_tech/wiki_tech.csv")

In [5]:
sums_pd.head()

,Original sentences
0,"['Technology (""science of craft"", from Greek τ..."
1,Technology can be the knowledge of techniques...
2,Systems (e
3,g
4,machines) applying technology by taking an in...


In [6]:
with open('stockholm/wikipedia_tech/wiki_tech.txt', 'r') as fp:
    text = fp.read().split('\n')

In [7]:
text

['[\'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives, such as scientific investigation',
 ' Technology can be the knowledge of techniques, processes, and the like, or it can be embedded in machines to allow for operation without detailed knowledge of their workings',
 ' Systems (e',
 'g',
 " machines) applying technology by taking an input, changing it according to the system\\'s use, and then producing an outcome are referred to as technology systems or technological systems",
 '\\nThe simplest form of technology is the development and use of basic tools',
 ' The prehistoric invention of shaped stone tools followed by the discovery of how to control fire increased sources of food',
 ' The later Neolithic Revolution extended this, and quadrupled the sustenance available from a territory',

In [8]:
# %%
import torch
import string

In [9]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [10]:
str1 = ''.join(text)

In [11]:
# !pip install -U spacy
import spacy

In [12]:
# Load small english model: https://spacy.io/models
nlp=spacy.load("en_core_web_sm")
nlp
#> spacy.lang.en.English at 0x7fd40c2eec50

In [13]:
my_doc = nlp(str1)
type(my_doc)

spacy.tokens.doc.Doc

In [14]:
my_doc

['Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives, such as scientific investigation Technology can be the knowledge of techniques, processes, and the like, or it can be embedded in machines to allow for operation without detailed knowledge of their workings Systems (eg machines) applying technology by taking an input, changing it according to the system\'s use, and then producing an outcome are referred to as technology systems or technological systems\nThe simplest form of technology is the development and use of basic tools The prehistoric invention of shaped stone tools followed by the discovery of how to control fire increased sources of food The later Neolithic Revolution extended this, and quadrupled the sustenance available from a territory  The invention of the wheel helped humans

In [15]:
for token in my_doc:
  print(token.text,'---- ',token.pos_)

Streaming output truncated to the last 5000 lines.
are ----  AUX
those ----  DET
technical ----  ADJ
innovations ----  NOUN
which ----  DET
represent ----  VERB
progressive ----  ADJ
developments ----  NOUN
within ----  ADP
a ----  DET
field ----  NOUN
for ----  ADP
competitive ----  ADJ
advantage ----  NOUN
; ----  PUNCT
converging ----  NOUN
technologies ----  NOUN
represent ----  VERB
previously ----  ADV
distinct ----  ADJ
fields ----  NOUN
which ----  DET
are ----  AUX
in ----  ADP
some ----  DET
way ----  NOUN
moving ----  VERB
towards ----  ADP
stronger ----  ADJ
inter ----  ADJ
- ----  NOUN
connection ----  ADJ
and ----  CCONJ
similar ----  ADJ
goals ----  NOUN
However ----  ADV
, ----  PUNCT
the ----  DET
opinion ----  NOUN
on ----  ADP
the ----  DET
degree ----  NOUN
of ----  ADP
the ----  DET
impact ----  NOUN
, ----  PUNCT
status ----  NOUN
and ----  CCONJ
economic ----  ADJ
viability ----  NOUN
of ----  ADP
several ----  ADJ
emerging ----  VERB
and ----  CCONJ
converging -

In [16]:
# Function to identify  if tokens are named entities and replace them with UNKNOWN
def remove_details(word):
  if word.pos_ =='ADJ':
    return ' <mask> '
  return word.string


# Function where each token of spacy doc is passed through remove_deatils()
def update_article(doc):
  # iterrating through all entities
  for ent in doc.ents:
    ent.merge()
  # Passing each token through remove_details() function.
  tokens = map(remove_details,doc)
  return ''.join(tokens)

# Passing our news_doc to the function update_article()
my_doc1 = update_article(my_doc)

In [17]:
my_doc1

'[\'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives,  <mask> as  <mask> investigation Technology can be the knowledge of techniques, processes, and the  <mask> , or it can be embedded in machines to allow for operation without  <mask> knowledge of their workings Systems (eg machines) applying technology by taking an input, changing it according to the system\\\'s use, and then producing an outcome are referred to as technology systems or  <mask> systems\\nThe  <mask> form of technology is the development and use of  <mask> tools The  <mask> invention of  <mask> stone tools followed by the discovery of how to control fire increased sources of food The  <mask> Neolithic Revolution extended this, and quadrupled the sustenance  <mask> from a territory  The invention of the wheel helped humans

In [18]:
my_doc2 = nlp(my_doc1)
type(my_doc2)

spacy.tokens.doc.Doc

In [19]:
sentences = list(my_doc2.sents)

In [20]:
len(sentences)

264

In [21]:
list_of_strings  = [i.text for i in sentences]

In [22]:
list_of_strings

['[\'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives,  <mask> as  <mask> investigation Technology can be the knowledge of techniques, processes, and the  <mask> , or it can be embedded in machines to allow for operation without  <mask> knowledge of their workings Systems (eg machines) applying technology by taking an input, changing it according to the system\\\'s use, and then producing an outcome are referred to as technology systems or  <mask>',
 'systems\\nThe  ',
 '<mask> form of technology is the development and use of  <mask> tools',
 'The  <mask> invention of  <mask> stone tools followed by the discovery of how to control fire increased sources of food',
 'The  <mask',
 '> Neolithic Revolution extended this, and quadrupled the sustenance  <mask> from a territory  ',
 'The inventio

In [23]:
from pandas import DataFrame
df = DataFrame (list_of_strings,columns=['sentence'])
df.head()

,sentence
0,"['Technology (""science of craft"", from Greek τ..."
1,systems\nThe
2,<mask> form of technology is the development a...
3,The <mask> invention of <mask> stone tools f...
4,The <mask


In [24]:
with open('wiki_tech_adj_masked.txt', 'w') as f:
  for item in df['sentence'].tolist():
        #import re
        #re.sub('[^a-zA-Z0-9]*$""', ' ', item)
        f.write("'%s',\n" % item)